In [1]:
from torch.utils.data import Dataset, DataLoader
import torch, torchvision
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable
import torchvision.transforms as transforms
import pandas as pd
import os
import glob
import numpy as np
import rasterio as rio
from rasterio.plot import reshape_as_image
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
test_directory = '/home/mcgrau/PycharmProjects/SS22_AIML/data/testset'
model_directory = '/home/mcgrau/PycharmProjects/SS22_AIML/best_models'
model_name_directory = 'resnet_50'
model_evaluated = os.path.join(model_directory, model_name_directory)
best_model_directory = os.path.join(model_evaluated, 'resnet_50_model_epoch_99.pth')

In [3]:
classes = [
    "AnnualCrop",
    "Forest",
    "HerbaceousVegetation",
    "Highway",
    "Industrial",
    "Pasture",
    "PermanentCrop",
    "Residential",
    "River",
    "SeaLake"
]

In [4]:
idx_to_class = {i:j for i, j in enumerate(classes)}
class_to_idx = {value:key for key,value in idx_to_class.items()}

In [5]:
import torchvision.models as models

base_model = models.resnet50(pretrained=True)

# modify first and final layer
base_model.conv1 = torch.nn.Conv2d(12, 64, kernel_size=(3, 3),
                              stride=(2, 2), padding=(3, 3), bias=False)
base_model.fc = torch.nn.Linear(4*512, 1024)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /home/mcgrau/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [6]:
class MyResNet(nn.Module):
    def __init__(self, my_pretrained_model):
        super(MyResNet, self).__init__()
        self.pretrained = my_pretrained_model
        self.my_new_layers = nn.Sequential(nn.Linear(1024, 512),
                                           nn.ReLU(inplace=True),
                                           nn.Linear(512, 10),
                                           nn.LogSoftmax(dim=1))

    def forward(self, x):
        x = self.pretrained(x)
        x = self.my_new_layers(x)
        return x

In [7]:
model = MyResNet(my_pretrained_model=base_model)
model

MyResNet(
  (pretrained): ResNet(
    (conv1): Conv2d(12, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [8]:
testSamples = glob.glob(os.path.join(test_directory, "*.npy"))
len(testSamples)

4232

In [9]:
transformer = transforms.Compose([
                                  transforms.ToTensor(),
                                  #transforms.Resize(224),
                                  transforms.Normalize(mean = [809.2, 900.5, 1061.4, 1091.7, 1384.5, 1917.8,
             2105.2, 2186.3, 2224.8, 2346.8, 1901.2, 1460.42],
                                                      std = [ 441.8, 624.7, 640.8, 718.1, 669.1, 767.5,
             843.3, 947.9, 882.4, 813.7, 716.9, 674.8])
])

In [19]:
class testDataset(Dataset):
    def __init__(self, test_directory, transform=None):
        self.files = glob.glob(os.path.join(test_directory, "*.npy"))
        self.transform = transform

    def __len__(self):
      return len(self.files)

    def __getitem__(self, idx):
        item = self.files[idx]
        image = np.load(item).astype(float)
        #image = reshape_as_image(image.astype(float))
        number = int(item.split('/')[-1].split('_')[1].split('.')[0])

        if self.transform:
          image = self.transform(image)

        return image, number

In [20]:
testData = testDataset(test_directory = test_directory, transform = transformer)

In [21]:
test = testData[1]
print(test)

(tensor([[[-1.1888, -1.1910, -1.1933,  ..., -1.2159, -1.2205, -1.2250],
         [-1.1842, -1.1888, -1.1910,  ..., -1.2159, -1.2205, -1.2250],
         [-1.1797, -1.1842, -1.1888,  ..., -1.2159, -1.2205, -1.2250],
         ...,
         [-1.2001, -1.2001, -1.1978,  ..., -1.2091, -1.2137, -1.2182],
         [-1.2024, -1.2001, -1.1978,  ..., -1.2137, -1.2182, -1.2205],
         [-1.2024, -1.1978, -1.1956,  ..., -1.2159, -1.2205, -1.2250]],

        [[-1.0573, -1.0445, -1.0573,  ..., -1.0573, -1.0653, -1.0381],
         [-0.9533, -0.9308, -0.9565,  ..., -1.0349, -1.0093, -1.0397],
         [-1.0525, -1.0701, -1.0941,  ..., -1.1181, -1.1229, -1.0781],
         ...,
         [-1.0845, -1.0237, -1.0429,  ..., -1.0461, -1.0077, -0.9933],
         [-1.0237, -0.9917, -1.0525,  ..., -1.0941, -1.0221, -1.0109],
         [-1.0493, -1.0909, -1.1197,  ..., -1.0285, -0.9757, -1.0685]],

        [[-1.1820, -1.1882, -1.2194,  ..., -1.2132, -1.1960, -1.1773],
         [-1.1507, -1.1289, -1.1664,  ..., -

In [22]:
test_loader = DataLoader(testData, shuffle=False)

In [23]:
# load state_dict from path
state_dict_best = torch.load(best_model_directory, map_location=torch.device('cpu'))

# init pre-trained model class
best_model = model

# load pre-trained models
best_model.load_state_dict(state_dict_best)

<All keys matched successfully>

In [24]:
# set model in evaluation mode
best_model.double()
best_model.eval()

MyResNet(
  (pretrained): ResNet(
    (conv1): Conv2d(12, 64, kernel_size=(3, 3), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
     

In [25]:
predictions = []
numbers = []

for i, (images, nums) in enumerate(test_loader):
    # run forward pass through the network
    pred = torch.argmax(best_model(images), dim=1)
    predictions.append(pred.int().item())
    numbers.append(nums.int().item())

In [26]:
print(len(predictions))

4232


In [27]:
predClasses = np.vectorize(idx_to_class.get)(predictions)

d = {'test_id': numbers, 'label': predClasses}
predData = pd.DataFrame(data = d)
predData = predData.sort_values(by=['test_id'])
print(predData.head(10))

      test_id                 label
2829        0  HerbaceousVegetation
2326        1  HerbaceousVegetation
3410        2            AnnualCrop
2701        3               SeaLake
3763        4  HerbaceousVegetation
3752        5            AnnualCrop
3271        6               SeaLake
1273        7               SeaLake
1029        8            AnnualCrop
3926        9            AnnualCrop


In [28]:
predData.to_csv(os.path.join(model_evaluated,'submission.csv'), index = False)

In [29]:
predData.describe()

,test_id
count,4232.000000
mean,2115.500000
std,1221.817499
min,0.000000
25%,1057.750000
50%,2115.500000
75%,3173.250000
max,4231.000000
